## 加载数据

数据已经预处理过了

In [1]:
import pandas as pd

# 加载数据
data = pd.read_csv('data_hh/result/pre_2023-2025_with_comp_train.csv', dtype={'flt_no': str})

# 查看前几行数据，确保加载成功
# 显示前两行数据以确保正确加载
print(data.shape)
print(data.head(5))
print(data.tail(5))

(1689609, 31)
   flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
0  3c6705  110.0      195     1       1      1.30   97  a9f275  24cc4b   
1  6a67d9  110.0      195     1       1      1.25   67  d56796  24cc4b   
2  de569b   93.0      190     3       3      4.75   22  d56796  ec1b13   
3  284078  162.0      320     1       1      1.83   55  e3d141  d618e3   
4  f8765b  167.0      320     1       1      3.98  166  9550bc  2e1ea1   

        c  ...  hour  minute      hour_sin      hour_cos  from   to  \
0  7f9c2b  ...    14      35 -5.000000e-01 -8.660254e-01   a9f  c4b   
1  7f9c2b  ...    22      40 -5.000000e-01  8.660254e-01   d56  c4b   
2  dcf90a  ...    18       0 -1.000000e+00 -1.836970e-16   d56  b13   
3  7f9c2b  ...    12      55  1.224647e-16 -1.000000e+00   e3d  8e3   
4  7f9c2b  ...    13      10 -2.588190e-01 -9.659258e-01   955  ea1   

   comp_price_diff  comp_price_diff_abs  comp_price_ratio  \
0       -94.900773            94.900773          0.20

In [2]:
# 检查标准化后的统计信息
print("\n标准化后的统计信息：")
print(data['pax'].describe())


标准化后的统计信息：
count    1.689609e+06
mean     1.222227e+02
std      5.222077e+01
min      2.000000e+01
25%      8.300000e+01
50%      1.280000e+02
75%      1.580000e+02
max      3.110000e+02
Name: pax, dtype: float64


## 编码分类变量

### 新增城市标签

In [3]:
import json
# 加载字典
with open('data_hh/result/encoder/city_labels_航班频率加权图标签.json', 'r') as file:
    city_labels_loaded = json.load(file)

print("加载的字典：", city_labels_loaded)

加载的字典： {'000': 1, '13f': 0, '8e3': 0, 'a18': 0, 'eda': 2, '01a': 1, 'b5a': 0, 'ea1': 0, '03f': 2, '1be': 0, '3bc': 0, '578': 0, '5df': 0, '5fb': 0, '760': 0, '792': 0, '7ff': 0, '90a': 0, '974': 0, 'bbb': 0, 'cb5': 0, 'ffe': 0, '059': 2, '022': 0, '077': 0, '0d7': 0, '0dd': 0, '1a3': 0, '1e9': 0, '1ed': 0, '25c': 0, '311': 0, '354': 0, '451': 0, '510': 0, '5d7': 0, '63a': 0, '666': 0, '6e3': 0, '7f1': 0, '812': 0, '8a1': 0, '9d4': 0, '9e9': 0, 'b3a': 0, 'bff': 0, 'c24': 0, 'c3a': 0, 'c4c': 0, 'c98': 0, 'c9f': 0, 'cb7': 0, 'd7c': 0, 'e76': 0, 'e9c': 0, 'efb': 0, '05c': 2, '141': 0, '157': 0, '709': 0, '8ac': 0, '97f': 0, '988': 0, 'a2f': 0, 'b09': 0, 'c14': 0, 'c4b': 0, 'e5b': 0, 'ec2': 0, '074': 1, '4d0': 0, '695': 0, 'c2c': 0, 'fa9': 0, '08c': 1, '08f': 2, '3c5': 0, '5e8': 0, 'a72': 0, 'd54': 0, 'ff1': 0, '0ac': 1, '0c8': 1, '0ca': 1, 'e09': 0, '0d1': 1, '13e': 0, '1ee': 0, '43e': 0, '0db': 2, '919': 0, 'd08': 0, '0ff': 2, '11b': 1, '123': 2, '5de': 0, '8eb': 0, '124': 1, '130': 2, '1

In [4]:
# 使用 map 对 'a', 'b', 'c', 'from', 'to' 列进行标签化，新增对应的标签列
data['a_label'] = data['a'].map(city_labels_loaded)
data['b_label'] = data['b'].map(city_labels_loaded)
data['c_label'] = data['c'].map(city_labels_loaded)
data['from_label'] = data['from'].map(city_labels_loaded)
data['to_label'] = data['to'].map(city_labels_loaded)

### 新增城市二维嵌入

In [5]:
import json

# 加载字典
with open('data_hh/result/encoder/城市嵌入编码_航班频率加权图.json', 'r') as file:
    city_embeddings = json.load(file)

print("加载的字典：", city_embeddings)


加载的字典： {'000': [-1.6952509880065918, -0.8259809613227844], '13f': [0.7161105871200562, -0.43590694665908813], '8e3': [0.5429399609565735, -0.520667314529419], 'a18': [0.20013581216335297, -0.4142817258834839], 'eda': [-0.15405425429344177, -1.1048771142959595], '01a': [-1.7130950689315796, 0.15070012211799622], 'b5a': [0.5836260318756104, -0.7250841856002808], 'ea1': [0.2437203973531723, 0.056170180439949036], '03f': [-1.0081822872161865, -0.733413577079773], '1be': [0.8263573050498962, -0.8512860536575317], '3bc': [1.4952945709228516, -1.2341231107711792], '578': [1.0146775245666504, -0.5335901975631714], '5df': [0.842978298664093, -0.9285436272621155], '5fb': [0.7287545204162598, -0.4136313796043396], '760': [1.2267078161239624, -0.6878513693809509], '792': [0.1306421011686325, -0.7094545364379883], '7ff': [0.9274259805679321, -0.45611515641212463], '90a': [0.06965449452400208, -0.2664845883846283], '974': [1.24435293674469, -1.1002029180526733], 'bbb': [0.4501871168613434, -0.740858

In [6]:
import pandas as pd
import numpy as np

# 将 city_embeddings 转换为 DataFrame
embedding_df = pd.DataFrame.from_dict(city_embeddings, orient='index', columns=['embedding_1', 'embedding_2'])
embedding_df.index.name = 'city'

# 用 'a', 'b', 'c', 'from', 'to' 字段与 embedding_df 合并
data = data.merge(embedding_df, left_on='a', right_index=True, how='left')
data.rename(columns={'embedding_1': 'a_embedding_1', 'embedding_2': 'a_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='b', right_index=True, how='left')
data.rename(columns={'embedding_1': 'b_embedding_1', 'embedding_2': 'b_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='c', right_index=True, how='left')
data.rename(columns={'embedding_1': 'c_embedding_1', 'embedding_2': 'c_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='from', right_index=True, how='left')
data.rename(columns={'embedding_1': 'from_embedding_1', 'embedding_2': 'from_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='to', right_index=True, how='left')
data.rename(columns={'embedding_1': 'to_embedding_1', 'embedding_2': 'to_embedding_2'}, inplace=True)

# 查看添加的新列
print(data[['a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 'to_embedding_1', 'to_embedding_2']])


         a_embedding_1  a_embedding_2  b_embedding_1  b_embedding_2  \
0                  NaN            NaN            NaN            NaN   
1                  NaN            NaN            NaN            NaN   
2                  NaN            NaN            NaN            NaN   
3                  NaN            NaN            NaN            NaN   
4                  NaN            NaN            NaN            NaN   
...                ...            ...            ...            ...   
1689604            NaN            NaN            NaN            NaN   
1689605            NaN            NaN            NaN            NaN   
1689606            NaN            NaN            NaN            NaN   
1689607            NaN            NaN            NaN            NaN   
1689608            NaN            NaN            NaN            NaN   

         c_embedding_1  c_embedding_2  from_embedding_1  from_embedding_2  \
0                  NaN            NaN         -1.641072          1.691

### 频率编码

使用 json 保存和加载 city_map

In [7]:
# 加载city_map
with open('data_hh/result/encoder/city_map_频率编码.json', 'r') as f:
    city_map = json.load(f)

# 使用 city_map 替换指定列的值
columns_to_replace = ['a', 'b', 'c', 'from', 'to']

# 遍历指定列并直接用 map 映射
for col in columns_to_replace:
    data[col] = data[col].map(city_map)


print(data)

         flt_no    cap aircraft  legs  leg_no  duration  pax   a   b   c  ...  \
0        3c6705  110.0      195     1       1      1.30   97 NaN NaN NaN  ...   
1        6a67d9  110.0      195     1       1      1.25   67 NaN NaN NaN  ...   
2        de569b   93.0      190     3       3      4.75   22 NaN NaN NaN  ...   
3        284078  162.0      320     1       1      1.83   55 NaN NaN NaN  ...   
4        f8765b  167.0      320     1       1      3.98  166 NaN NaN NaN  ...   
...         ...    ...      ...   ...     ...       ...  ...  ..  ..  ..  ...   
1689604  2f5ac7  161.0      738     3       2      2.58  128 NaN NaN NaN  ...   
1689605  8b52d8  161.0      738     1       1      2.60  152 NaN NaN NaN  ...   
1689606  cac556  161.0      738     1       1      1.77  149 NaN NaN NaN  ...   
1689607  0bd968  188.0      7MZ     1       1      2.40  144 NaN NaN NaN  ...   
1689608  2befc2  161.0      738     1       1      2.48  142 NaN NaN NaN  ...   

         a_embedding_1  a_e

### 'flt_no', 'bd_type', 'aircraft'编码

In [8]:
import joblib
from sklearn.preprocessing import LabelEncoder
import os

# 定义需要编码的分类特征
# categorical_columns = ['flt_no', 'bd_type', 'aircraft']
categorical_columns = ['flt_no', 'aircraft']

# 从保存的文件中加载编码器并应用到data
for col in categorical_columns:
    # 加载编码器
    encoder_path = os.path.join('data_hh/result/encoder/', f"{col}_encoder_all.pkl")
    le = joblib.load(encoder_path)
    
    try:
        # 对data进行转换
        data[col] = le.transform(data[col])
        print(f"{col}列编码完成")
    except ValueError as e:
        # 如果遇到新的类别，打印错误信息
        print(f"{col}列编码出错: {str(e)}")
        # 找出新的类别
        new_categories = set(data[col]) - set(le.classes_)
        print(f"{col}列中的新类别: {new_categories}")

# 查看编码后的结果
print("\n编码后的前几行数据：")
print(data[categorical_columns].head())

flt_no列编码完成
aircraft列编码完成

编码后的前几行数据：
   flt_no  aircraft
0    1104         2
1    1872         2
2    3916         0
3     721         9
4    4356         9


## 特征和目标分离
我们要预测的是pax字段，其他字段作为特征。

In [9]:
# 特征列
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft',  'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','holiday', 'hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label']]

# 有abc，有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
# X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
import numpy as np
# 存在性检查→缺则补（只在缺失时计算）
_need = ['quarter','is_weekend','is_holiday_season','hour_sin','hour_cos','month_sin','month_cos']
_missing = [c for c in _need if c not in data.columns]
if _missing:
    assert all(col in data.columns for col in ['month','weekday','hour']), '缺少 month/weekday/hour 无法补时间特征'
    data['quarter'] = data['month'].apply(lambda x: (x - 1) // 3 + 1)
    data['is_weekend'] = data['weekday'].isin([5, 6]).astype(int)
    data['is_holiday_season'] = data['month'].isin([1, 2, 7, 8, 10]).astype(int)
    data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
    data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
    data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

# 新增：竞品价格差异比例特征（相对当前航班）
if 'competitor_price' in data.columns and 'unit_price' in data.columns:
    _diff = data['competitor_price'] - data['unit_price']
    data['comp_price_diff_abs'] = _diff.abs()
    _den = data['unit_price'].replace(0, float('nan')).astype(float)
    data['comp_price_ratio'] = (_diff / _den).replace([np.inf, -np.inf], np.nan).fillna(0.0)
    data['comp_price_ratio_abs'] = data['comp_price_ratio'].abs()

X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday', 'hour', 'minute', 'quarter', 'is_weekend', 'is_holiday_season', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'from', 'to', 'unit_price', 'competitor_price', 'comp_price_diff_abs', 'comp_price_ratio', 'a_label', 'b_label', 'c_label', 'from_label', 'to_label', 'a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 'to_embedding_1', 'to_embedding_2']]

# 删除了abc，但有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]

# 目标列
y = data['pax']

### 对x进行标准化

In [10]:
from sklearn.preprocessing import StandardScaler

# 对所有特征进行标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 将标准化后的数据转回 DataFrame 格式
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# 查看结果
pd.set_option('display.max_columns', None)  # 显示所有列
print(X_scaled.head(1))


# 保存 scaler
joblib.dump(scaler, 'data_hh/result/encoder/standard_scaler_x.pkl')
print("x的标准化器已保存为 standard_scaler_x.pkl")

/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


     flt_no       cap  aircraft     legs    leg_no  duration   a   b   c  \
0 -0.848829 -1.494606 -1.378712 -0.85182 -0.558079 -1.236975 NaN NaN NaN   

       year     month      day   weekday      hour    minute   quarter  \
0 -0.977504 -1.307816 -1.68186  1.482253 -0.001571  0.479603 -1.051653   

   is_weekend  is_holiday_season  hour_sin  hour_cos  month_sin  month_cos  \
0    1.564266           1.333616 -0.481549 -0.919969   0.368371   1.241001   

       from        to  unit_price  competitor_price  comp_price_diff_abs  \
0 -1.350272  0.203254   -0.567115         -0.203918             -0.29615   

   comp_price_ratio  a_label  b_label  c_label  from_label  to_label  \
0          0.371132      NaN      NaN      NaN   -3.677972  -0.20304   

   a_embedding_1  a_embedding_2  b_embedding_1  b_embedding_2  c_embedding_1  \
0            NaN            NaN            NaN            NaN            NaN   

   c_embedding_2  from_embedding_1  from_embedding_2  to_embedding_1  \
0         

### 对y进行标准化

In [11]:
# 对目标列 y 进行标准化
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))  # 将 y 转换为 2D 数组进行标准化

# 转换回 DataFrame 格式
y_scaled = pd.DataFrame(y_scaled, columns=['pax_scaled'])

# 查看标准化后的 y
print(y_scaled.head())

# 保存 y 的 scaler
joblib.dump(scaler_y, 'data_hh/result/encoder/standard_scaler_y.pkl')
print("y的标准化器已保存为 standard_scaler_y.pkl")

   pax_scaled
0   -0.483002
1   -1.057486
2   -1.919213
3   -1.287280
4    0.838312
y的标准化器已保存为 standard_scaler_y.pkl


## 训练XGBoost模型

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 自定义 SMAPE 函数
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred)
    return np.mean(diff / denominator) * 100

# 自定义评估函数
def smape_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    smape_value = smape(y_true, y_pred)
    return 'SMAPE', smape_value

# 数据划分
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 输出数据集大小
print(f'训练集大小: {X_train.shape[0]}')
print(f'验证集大小: {X_val.shape[0]}')
print(f'测试集大小: {X_test.shape[0]}')

# 转换为 DMatrix 格式
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# 使用贝叶斯优化替代手动参数设置
from xgb_bayes_opt import bayesian_tune_xgb, smape_eval

base_params = {
    'objective': 'reg:squarederror',
    # 可选: 'tree_method': 'hist', 'nthread': 8, 'seed': 42
}

# 评估集（以验证集为优化目标）
evals = [(dtrain, 'train'), (dval, 'validation')]

model, best_params, best_smape = bayesian_tune_xgb(
    dtrain,
    evals,
    base_params=base_params,
    init_points=16,
    n_iter=50,
    early_stopping_rounds=10,
    verbose_eval=10,
    save_model_name='xgb_model_bayes.json',
)

print('Best SMAPE:', best_smape)
print('Best Params:', best_params)

# 预测测试集
y_pred = model.predict(dtest)

# # 测试集 SMAPE 评估
# test_smape = smape(y_test, y_pred)
# print(f'SMAPE on Test Set: {test_smape:.2f}%')

# # 测试集 MSE 评估
# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error on Test Set: {mse}')

训练集大小: 1351687
验证集大小: 168961
测试集大小: 168961
|   iter    |  target   | learni... | max_depth | subsample | colsam... |   alpha   |  lambda_  | num_bo... |
-------------------------------------------------------------------------------------------------------------


/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-rmse:0.90942	train-SMAPE:163.16492	validation-rmse:0.90752	validation-SMAPE:163.23141
[10]	train-rmse:0.48956	train-SMAPE:79.58442	validation-rmse:0.49372	validation-SMAPE:80.33363
[20]	train-rmse:0.41069	train-SMAPE:67.72217	validation-rmse:0.42016	validation-SMAPE:68.86636
[30]	train-rmse:0.38657	train-SMAPE:64.07450	validation-rmse:0.40022	validation-SMAPE:65.56738
[40]	train-rmse:0.37243	train-SMAPE:61.92568	validation-rmse:0.39000	validation-SMAPE:63.84087
[50]	train-rmse:0.36374	train-SMAPE:60.69254	validation-rmse:0.38438	validation-SMAPE:62.93590
[60]	train-rmse:0.35664	train-SMAPE:59.71617	validation-rmse:0.38001	validation-SMAPE:62.25265
[70]	train-rmse:0.35244	train-SMAPE:59.15720	validation-rmse:0.37755	validation-SMAPE:61.89932
[80]	train-rmse:0.34692	train-SMAPE:58.41180	validation-rmse:0.37432	validation-SMAPE:61.39607
[90]	train-rmse:0.34066	train-SMAPE:57.56782	validation-rmse:0.37061	validation-SMAPE:60.81473
[100]	train-rmse:0.33614	train-SMAPE:56.96400	val

[120]	train-rmse:0.48024	train-SMAPE:73.95606	validation-rmse:0.48096	validation-SMAPE:74.29800
[130]	train-rmse:0.47839	train-SMAPE:73.71692	validation-rmse:0.47912	validation-SMAPE:74.06760
[140]	train-rmse:0.47684	train-SMAPE:73.50990	validation-rmse:0.47759	validation-SMAPE:73.85664
[150]	train-rmse:0.47541	train-SMAPE:73.31390	validation-rmse:0.47616	validation-SMAPE:73.66500
[160]	train-rmse:0.47424	train-SMAPE:73.17521	validation-rmse:0.47502	validation-SMAPE:73.54153
[170]	train-rmse:0.47302	train-SMAPE:73.01460	validation-rmse:0.47383	validation-SMAPE:73.38522
[180]	train-rmse:0.47191	train-SMAPE:72.85410	validation-rmse:0.47274	validation-SMAPE:73.23422
[190]	train-rmse:0.47096	train-SMAPE:72.75368	validation-rmse:0.47178	validation-SMAPE:73.13223
[200]	train-rmse:0.46962	train-SMAPE:72.64277	validation-rmse:0.47046	validation-SMAPE:73.03287
[210]	train-rmse:0.46834	train-SMAPE:72.46742	validation-rmse:0.46918	validation-SMAPE:72.85622
[220]	train-rmse:0.46724	train-SMAPE:72.

[130]	train-rmse:0.46764	train-SMAPE:72.80913	validation-rmse:0.46863	validation-SMAPE:73.20408
[140]	train-rmse:0.46601	train-SMAPE:72.60495	validation-rmse:0.46706	validation-SMAPE:72.98158
[150]	train-rmse:0.46449	train-SMAPE:72.37573	validation-rmse:0.46557	validation-SMAPE:72.75878
[160]	train-rmse:0.46288	train-SMAPE:72.18362	validation-rmse:0.46399	validation-SMAPE:72.56721
[170]	train-rmse:0.46167	train-SMAPE:72.01231	validation-rmse:0.46281	validation-SMAPE:72.40294
[180]	train-rmse:0.46036	train-SMAPE:71.83990	validation-rmse:0.46155	validation-SMAPE:72.24147
[190]	train-rmse:0.45925	train-SMAPE:71.69406	validation-rmse:0.46049	validation-SMAPE:72.10223
[200]	train-rmse:0.45815	train-SMAPE:71.56992	validation-rmse:0.45940	validation-SMAPE:71.97534
[210]	train-rmse:0.45697	train-SMAPE:71.43453	validation-rmse:0.45827	validation-SMAPE:71.84011
[220]	train-rmse:0.45578	train-SMAPE:71.29838	validation-rmse:0.45709	validation-SMAPE:71.69744
[230]	train-rmse:0.45453	train-SMAPE:71.

...

/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-rmse:0.77308	train-SMAPE:128.64528	validation-rmse:0.77252	validation-SMAPE:128.94431
[10]	train-rmse:0.39162	train-SMAPE:64.62600	validation-rmse:0.40693	validation-SMAPE:66.20338
[20]	train-rmse:0.36513	train-SMAPE:60.84287	validation-rmse:0.38914	validation-SMAPE:63.30927
[30]	train-rmse:0.34806	train-SMAPE:58.47046	validation-rmse:0.37850	validation-SMAPE:61.61544
[40]	train-rmse:0.33718	train-SMAPE:57.00317	validation-rmse:0.37305	validation-SMAPE:60.74978
[50]	train-rmse:0.32698	train-SMAPE:55.66765	validation-rmse:0.36815	validation-SMAPE:60.01374
[60]	train-rmse:0.31657	train-SMAPE:54.34567	validation-rmse:0.36397	validation-SMAPE:59.36565
[70]	train-rmse:0.30806	train-SMAPE:53.20973	validation-rmse:0.36112	validation-SMAPE:58.95601
[80]	train-rmse:0.30060	train-SMAPE:52.24384	validation-rmse:0.35891	validation-SMAPE:58.58672
[90]	train-rmse:0.29386	train-SMAPE:51.38848	validation-rmse:0.35730	validation-SMAPE:58.36261
[100]	train-rmse:0.28893	train-SMAPE:50.75033	val

/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-rmse:0.77687	train-SMAPE:128.90465	validation-rmse:0.77562	validation-SMAPE:129.11118
[10]	train-rmse:0.40463	train-SMAPE:65.89574	validation-rmse:0.41349	validation-SMAPE:66.95879
[20]	train-rmse:0.38257	train-SMAPE:62.58260	validation-rmse:0.39636	validation-SMAPE:64.15936
[30]	train-rmse:0.37125	train-SMAPE:61.05675	validation-rmse:0.38817	validation-SMAPE:62.90465
[40]	train-rmse:0.36089	train-SMAPE:59.73619	validation-rmse:0.38114	validation-SMAPE:61.93476
[50]	train-rmse:0.35066	train-SMAPE:58.35341	validation-rmse:0.37449	validation-SMAPE:60.94560
[60]	train-rmse:0.34506	train-SMAPE:57.64302	validation-rmse:0.37102	validation-SMAPE:60.45033
[70]	train-rmse:0.33860	train-SMAPE:56.81078	validation-rmse:0.36772	validation-SMAPE:59.94219
[80]	train-rmse:0.33264	train-SMAPE:56.05031	validation-rmse:0.36462	validation-SMAPE:59.48157
[90]	train-rmse:0.32675	train-SMAPE:55.33339	validation-rmse:0.36189	validation-SMAPE:59.12739
[100]	train-rmse:0.32222	train-SMAPE:54.76353	val

[400]	train-rmse:0.32639	train-SMAPE:55.39770	validation-rmse:0.35869	validation-SMAPE:58.96243
[410]	train-rmse:0.32520	train-SMAPE:55.24377	validation-rmse:0.35808	validation-SMAPE:58.88047
[420]	train-rmse:0.32408	train-SMAPE:55.09257	validation-rmse:0.35754	validation-SMAPE:58.79525
[430]	train-rmse:0.32312	train-SMAPE:54.97517	validation-rmse:0.35711	validation-SMAPE:58.73154
[440]	train-rmse:0.32199	train-SMAPE:54.83116	validation-rmse:0.35666	validation-SMAPE:58.66500
[450]	train-rmse:0.32103	train-SMAPE:54.71173	validation-rmse:0.35626	validation-SMAPE:58.60532
[460]	train-rmse:0.32002	train-SMAPE:54.58076	validation-rmse:0.35580	validation-SMAPE:58.54255
[470]	train-rmse:0.31893	train-SMAPE:54.45065	validation-rmse:0.35529	validation-SMAPE:58.46776
[480]	train-rmse:0.31807	train-SMAPE:54.34330	validation-rmse:0.35495	validation-SMAPE:58.41013
[490]	train-rmse:0.31724	train-SMAPE:54.23753	validation-rmse:0.35459	validation-SMAPE:58.36039
[500]	train-rmse:0.31636	train-SMAPE:54.

In [13]:
y_pred

array([-0.42942294,  0.5920659 , -1.0133168 , ...,  0.84471774,
        0.9610513 , -0.43613452], dtype=float32)

In [14]:
# 反标准化 y_test
y_test_original = scaler_y.inverse_transform(y_test.values.reshape(-1, 1))

# 反标准化预测结果 y_pred
y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

In [15]:
y_test_original

array([[ 92.],
       [158.],
       [ 60.],
       ...,
       [178.],
       [170.],
       [ 96.]])

In [16]:
y_pred_original

array([[ 99.797935],
       [153.14085 ],
       [ 69.306564],
       ...,
       [166.33452 ],
       [172.40955 ],
       [ 99.44746 ]], dtype=float32)

In [17]:
# 正确的写法
test_results = list(zip(y_test_original[:100], y_pred_original[:100]))  # 真实值和预测值

print("\n20条测试结果（真实值 vs 预测值）:")
for i, (true_value, pred_value) in enumerate(test_results[:40]):
    # 如果是多维数组，使用 .item() 转换为标量
    true_value = true_value.item() if isinstance(true_value, np.ndarray) else true_value
    pred_value = pred_value.item() if isinstance(pred_value, np.ndarray) else pred_value
    print(f"第{i+1}条: 真实值={true_value}, 预测值={pred_value:.2f}")



20条测试结果（真实值 vs 预测值）:
第1条: 真实值=92.0, 预测值=99.80
第2条: 真实值=158.0, 预测值=153.14
第3条: 真实值=60.0, 预测值=69.31
第4条: 真实值=216.0, 预测值=207.28
第5条: 真实值=158.0, 预测值=147.89
第6条: 真实值=174.0, 预测值=172.42
第7条: 真实值=183.0, 预测值=168.70
第8条: 真实值=61.0, 预测值=88.99
第9条: 真实值=89.0, 预测值=125.90
第10条: 真实值=161.0, 预测值=151.59
第11条: 真实值=170.0, 预测值=161.79
第12条: 真实值=166.0, 预测值=157.65
第13条: 真实值=131.0, 预测值=118.55
第14条: 真实值=119.0, 预测值=123.30
第15条: 真实值=85.0, 预测值=85.41
第16条: 真实值=134.0, 预测值=103.69
第17条: 真实值=172.0, 预测值=147.57
第18条: 真实值=75.0, 预测值=60.68
第19条: 真实值=73.0, 预测值=83.37
第20条: 真实值=26.0, 预测值=41.71
第21条: 真实值=37.0, 预测值=42.58
第22条: 真实值=129.0, 预测值=129.96
第23条: 真实值=54.0, 预测值=35.97
第24条: 真实值=129.0, 预测值=135.02
第25条: 真实值=70.0, 预测值=71.83
第26条: 真实值=169.0, 预测值=166.65
第27条: 真实值=101.0, 预测值=82.77
第28条: 真实值=31.0, 预测值=62.59
第29条: 真实值=74.0, 预测值=72.95
第30条: 真实值=91.0, 预测值=115.39
第31条: 真实值=140.0, 预测值=146.75
第32条: 真实值=70.0, 预测值=62.35
第33条: 真实值=74.0, 预测值=63.80
第34条: 真实值=83.0, 预测值=93.66
第35条: 真实值=125.0, 预测值=114.78
第36条: 真实值=156.0, 预测值=146.98
第37条: 真实值=14

In [18]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_smape(y_true, y_pred):
    """
    计算 Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_pred - y_true)
    
    # 避免除以零，将分母中为零的项替换为一个小值
    denominator = np.where(denominator == 0, 1e-8, denominator)
    
    smape = 100 * np.mean(diff / denominator)
    return smape


def calculate_mape(y_true, y_pred):
    """
    计算 Mean Absolute Percentage Error (MAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    
    # 避免除以零，将 y_true 中的零值替换为一个小值
    y_true = np.where(y_true == 0, 1e-8, y_true)
    
    mape = 100 * np.mean(np.abs((y_true - y_pred) / y_true))
    return mape


# 假设 y_test 和 y_pred 已经是标准化反归一化后的数据
# 将其转换为一维数组以确保形状一致
y_test = np.array(y_test_original).ravel()
y_pred = np.array(y_pred_original).ravel()

# 评估指标
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = calculate_mape(y_test, y_pred)
smape = calculate_smape(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 打印结果
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')
print(f'Symmetric Mean Absolute Percentage Error (SMAPE): {smape:.4f}%')
print(f'R-squared (R²): {r2:.4f}')

Mean Squared Error (MSE): 321.3528
Root Mean Squared Error (RMSE): 17.9263
Mean Absolute Error (MAE): 13.4265
Mean Absolute Percentage Error (MAPE): 15.3634%
Symmetric Mean Absolute Percentage Error (SMAPE): 13.8559%
R-squared (R²): 0.8822


似乎对于较小值预测存在误差

## 保存模型

In [19]:
model.save_model("data_hh/result/model/归一化_xgboost_model_1000.json")
print("模型已保存为 xgboost_model_1000.json")

模型已保存为 xgboost_model_1000.json


## 超参数设置

## 不同特征重要程度测试

In [20]:
import xgboost as xgb
import matplotlib.pyplot as plt

# 假设 model 是训练好的 XGBoost 模型
xgb.plot_importance(model, importance_type='weight', title="Feature Importance (Weight)", height=0.5)
plt.show()

xgb.plot_importance(model, importance_type='gain', title="Feature Importance (Gain)", height=0.5)
plt.show()

xgb.plot_importance(model, importance_type='cover', title="Feature Importance (Cover)", height=0.5)
plt.show()

TypeError: can only be called with ndarray object

Error in callback <function _draw_all_if_interactive at 0x7f8e1c20dab0> (for post_execute), with arguments args (),kwargs {}:


ValueError: object __array__ method not producing an array

ImportError: cannot import name 'ERR_IGNORE' from 'numpy.core.umath' (/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/numpy/core/umath.py)

<Figure size 640x480 with 1 Axes>